# Importing

In [1]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [2]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3)
)

In [4]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [5]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 16813124 (64.14 MB)
Trainable params: 16813124 (64.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
conv_base.trainable = False

In [8]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory='../BrainTumor/Training',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150),
    # shuffle=True,
    # seed=42,
    # subset='training'  # Use this subset for training
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='../BrainTumor/Testing',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150),
    # shuffle=True,
    # seed=42,
    # subset='validation'  # Use this subset for validation
)

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.


In [9]:
# Normalize
def process(image,label):
    image = tensorflow.cast(image/255. ,tensorflow.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [12]:
history = model.fit(train_ds,epochs=15,validation_data=validation_ds)

Epoch 1/15
90/90 [==============================] - 186s 2s/step - loss: 0.0170 - accuracy: 0.9986 - val_loss: 2.3384 - val_accuracy: 0.7741
Epoch 2/15
90/90 [==============================] - 240s 3s/step - loss: 0.0185 - accuracy: 0.9983 - val_loss: 2.5095 - val_accuracy: 0.7690
Epoch 3/15
90/90 [==============================] - 335s 4s/step - loss: 0.0197 - accuracy: 0.9965 - val_loss: 2.5501 - val_accuracy: 0.7665
Epoch 4/15
90/90 [==============================] - 179s 2s/step - loss: 0.0747 - accuracy: 0.9746 - val_loss: 1.9975 - val_accuracy: 0.7690
Epoch 5/15
90/90 [==============================] - 301s 3s/step - loss: 0.0299 - accuracy: 0.9875 - val_loss: 2.5977 - val_accuracy: 0.7487
Epoch 6/15
90/90 [==============================] - 679s 8s/step - loss: 0.0088 - accuracy: 0.9993 - val_loss: 2.9399 - val_accuracy: 0.7589
Epoch 7/15
90/90 [==============================] - 173s 2s/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 2.9680 - val_accuracy: 0.7589
Epoch 8/15
90